In [1]:
import math

import numpy as np

from sktime.annotation.hidalgo import Hidalgo

# generate dataset
N = 10
N_half = int(N / 2)
np.random.seed(10002)
X = np.zeros((N, 6))

# half the points from one generating regime
for j in range(1):
    X[:N_half, j] = np.random.normal(0, 3, N_half)

# the other half from another
for j in range(3):
    X[N_half:, j] = np.random.normal(2, 1, N_half)

In [2]:
# get model
K = 2
model = Hidalgo(K=K, seed=0)

In [3]:
model.fit(X)

In [4]:
model.d_

array([0.8958701 , 2.00204128])

In [5]:
model.p_

array([0.66415462, 0.33584538])

In [6]:
model.Z

array([1, 1, 1, 1, 1, 2, 2, 2, 2, 1], dtype=int64)